# Lab 12: Path Planning and Execution
<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2025-05-09 14:52:58,689 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
import re
import numpy as np
import time

sensor_data_index = 0
sensor_data = [[0.0 for _ in range(1)] for _ in range(18)]
msg = None

def data_collect(uuid, byte_array):
    global sensor_data_index
    global sensor_data
    msg = ble.bytearray_to_string(byte_array)
    data = re.findall(r'-?\d+\.\d+', msg)
    if sensor_data_index < len(sensor_data):
        sensor_data[sensor_data_index] = [float(num) for num in data]
        sensor_data_index += 1

class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")
    
    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        global sensor_data, sensor_data_index 
        sensor_data_index = 0
        sensor_data = [[0.0 for _ in range(1)] for _ in range(18)]
        
        self.ble.start_notify(self.ble.uuid['RX_STRING'], data_collect)
    
        # Send command to rotate + scan
        self.ble.send_command(CMD.LOCALIZATION, f"60|3|0.005|200|1.28|100|95|800")
    
        # Wait for data to finish collecting (tune based on your duration)
        import asyncio
        asyncio.run(asyncio.sleep(20))
    
        self.ble.stop_notify(self.ble.uuid['RX_STRING'])
    
        sensor_ranges = np.array(sensor_data) / 1000.0
        sensor_bearings = np.array([])[np.newaxis].T
        
        return sensor_ranges, sensor_bearings

In [4]:
ble.disconnect()

NameError: name 'ble' is not defined

In [5]:
ble = get_ble_controller()
ble.connect()

2025-05-09 14:54:06,198 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:c3:e4:25:23:51
2025-05-09 14:54:10,817 | INFO     |: Connected to c0:c3:e4:25:23:51


C:\Users\qijin\Documents\Ece4160\IvannnHuang.github.io\Lab12\notebooks\base_ble.py:104: FutureWarning: This method will be removed future version, pass the callback to the BleakClient constructor instead.
  self.client.set_disconnected_callback(self.disconnect_handler)


In [6]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-05-09 14:54:11,893 | INFO     |:  | Number of observations per grid cell: 18
2025-05-09 14:54:11,900 | INFO     |:  | Precaching Views...


C:\Users\qijin\Documents\Ece4160\IvannnHuang.github.io\Lab12\localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2025-05-09 14:54:14,410 | INFO     |:  | Precaching Time: 2.510 secs
2025-05-09 14:54:14,411 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 14:54:14,412 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [9]:
cmdr.reset_plotter()
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
loc.plot_update_step_data(plot_data=True)

2025-05-09 12:31:40,214 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 12:31:40,215 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[1.335299]
 [1.994962]]
2025-05-09 12:32:00,697 | INFO     |: Update Step
2025-05-09 12:32:00,702 | INFO     |:      | Update Time: 0.002 secs
2025-05-09 12:32:00,704 | INFO     |: Bel index     : (np.int64(1), np.int64(0), np.int64(9)) with prob = 0.9999999
2025-05-09 12:32:00,706 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 12:32:00,709 | INFO     |: Belief        : (-1.219, -1.219, 10.000)


In [10]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.005|150|1.5|-4|-3|-2|-1|45|100|90|1.5|200|350|4.6") 

In [11]:
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
loc.plot_update_step_data(plot_data=True)

2025-05-09 12:32:01,078 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 12:32:01,079 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[2.800532]
 [1.939277]]
2025-05-09 12:32:28,479 | INFO     |: Update Step
2025-05-09 12:32:28,485 | INFO     |:      | Update Time: 0.004 secs
2025-05-09 12:32:28,489 | INFO     |: Bel index     : (np.int64(2), np.int64(2), np.int64(9)) with prob = 0.9999999
2025-05-09 12:32:28,492 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 12:32:28,495 | INFO     |: Belief        : (-0.914, -0.610, 10.000)


In [28]:
cmdr.reset_plotter()
pos = [[-1.219, -1.219, 10.000], [-0.91400, -0.61, 10.000], [0.30500, -0.305, -30.000], [0.610, -0.914, -10.000], [1.524, -0.914, -10.000], [1.829, -0.305, 10.000], [1.524, 0.810, -30.000], [0.000, 1.219, 10.000], [-0.305, -0.305, -10.000]]
pos1 = [[-1.2, -0.9, 10.000], [-0.900, -0.25, 10.000], [0.32500, -0.305, -30.000], [0.7, -0.95, -10], [1.589, -0.914, -10.000], [1.594, -0.554, -10.000], [1.554, 0.955, 130.000], [-0.050, 1.219, 10.000], [-0.235, -0.210, -10.000]]
loc.plot_pose(pos[0], pos[1], pos[2], pos1[0], pos1[1], pos1[2])
loc.plot_pose(pos[3], pos[4], pos[5], pos1[3], pos1[4], pos1[5])
loc.plot_pose(pos[6], pos[7], pos[8], pos1[6], pos1[7], pos1[8])

In [26]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.005|150|1.5|-2|-1|1|-1|45|100|90|1.5|200|300|4.6")  
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
loc.plot_update_step_data(plot_data=True)

Exception: Not connected to a BLE device

In [13]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.005|150|1.5|{x1}|{y1]|2|-3|45|100|90|1.5|200|500|4.6")  
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
[x1, y1] = loc.plot_update_step_data(plot_data=True)  

2025-05-09 14:16:32,449 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 14:16:32,450 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[1.298426]
 [1.451109]]
2025-05-09 14:17:01,851 | INFO     |: Update Step
2025-05-09 14:17:01,853 | INFO     |:      | Update Time: 0.001 secs
2025-05-09 14:17:01,854 | INFO     |: Bel index     : (np.int64(7), np.int64(1), np.int64(8)) with prob = 1.0
2025-05-09 14:17:01,855 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 14:17:01,856 | INFO     |: Belief        : (0.610, -0.914, -10.000)


In [20]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.005|150|1.5|{x1}|{y1}|5|-3|45|100|90|1.5|200|500|4.6")  
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
[x1, y1] = loc.plot_update_step_data(plot_data=True)  

2025-05-09 14:20:37,861 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 14:20:37,862 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[0.370818]
 [0.371576]]
2025-05-09 14:21:04,565 | INFO     |: Update Step
2025-05-09 14:21:04,571 | INFO     |:      | Update Time: 0.004 secs
2025-05-09 14:21:04,573 | INFO     |: Bel index     : (np.int64(10), np.int64(1), np.int64(8)) with prob = 0.9999999
2025-05-09 14:21:04,576 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 14:21:04,578 | INFO     |: Belief        : (1.524, -0.914, -10.000)


In [21]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.005|150|1.5|5|-3|5|-2|45|100|90|1.5|200|300|4.6")  
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
loc.plot_update_step_data(plot_data=True)

2025-05-09 14:21:09,363 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 14:21:09,364 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[0.411169]
 [0.389452]]
2025-05-09 14:21:36,185 | INFO     |: Update Step
2025-05-09 14:21:36,187 | INFO     |:      | Update Time: 0.001 secs
2025-05-09 14:21:36,188 | INFO     |: Bel index     : (np.int64(11), np.int64(3), np.int64(9)) with prob = 0.9999999
2025-05-09 14:21:36,189 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 14:21:36,190 | INFO     |: Belief        : (1.829, -0.305, 10.000)


In [22]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.005|150|1.5|5|-2|5|3|45|100|90|1.5|200|520|4.6")  
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
loc.plot_update_step_data(plot_data=True)

2025-05-09 14:21:42,845 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 14:21:42,846 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[0.345092]
 [0.29114 ]]
2025-05-09 14:22:12,008 | INFO     |: Update Step
2025-05-09 14:22:12,013 | INFO     |:      | Update Time: 0.003 secs
2025-05-09 14:22:12,015 | INFO     |: Bel index     : (np.int64(10), np.int64(6), np.int64(7)) with prob = 1.0
2025-05-09 14:22:12,016 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 14:22:12,017 | INFO     |: Belief        : (1.524, 0.610, -30.000)


In [8]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.008|250|1.5|5|3|0|3|45|95|90|1.5|200|520|3.9")  
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
loc.plot_update_step_data(plot_data=True)

2025-05-09 14:55:11,313 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 14:55:11,314 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[0.831375]
 [0.467727]]
2025-05-09 14:55:40,412 | INFO     |: Update Step
2025-05-09 14:55:40,424 | INFO     |:      | Update Time: 0.001 secs
2025-05-09 14:55:40,425 | INFO     |: Bel index     : (np.int64(5), np.int64(8), np.int64(9)) with prob = 1.0
2025-05-09 14:55:40,426 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 14:55:40,427 | INFO     |: Belief        : (0.000, 1.219, 10.000)


In [18]:
ble = get_ble_controller()
ble.connect()

2025-05-09 15:00:55,408 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:c3:e4:25:23:51
2025-05-09 15:01:03,448 | INFO     |: Connected to c0:c3:e4:25:23:51


In [19]:
ble.send_command(CMD.NAVIGATION, f"20|0.04|0.4|100|3|0.005|150|1.5|0|3|0|0|45|100|90|1.5|200|500|4.6")  
loc.init_grid_beliefs()
loc.get_observation_data()
loc.update_step()
loc.plot_update_step_data(plot_data=True)

2025-05-09 15:01:07,962 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-09 15:01:07,963 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[2.083969]
 [1.370472]]
2025-05-09 15:01:36,942 | INFO     |: Update Step
2025-05-09 15:01:36,944 | INFO     |:      | Update Time: 0.002 secs
2025-05-09 15:01:36,946 | INFO     |: Bel index     : (np.int64(4), np.int64(3), np.int64(8)) with prob = 0.9994511
2025-05-09 15:01:36,947 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-05-09 15:01:36,948 | INFO     |: Belief        : (-0.305, -0.305, -10.000)
2025-05-09 15:01:53,887 | INFO     |: Disconnected from C0:C3:E4:25:23:51


In [12]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-24 17:13:59,488 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-24 17:13:59,489 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[2.853784]
 [2.154413]
 [2.43498 ]
 [2.502618]
 [1.184313]
 [0.694924]
 [0.71937 ]
 [0.836877]
 [0.867118]
 [0.754732]
 [0.681533]
 [0.687442]
 [0.760213]
 [0.90022 ]
 [0.803055]
 [0.671762]
 [0.646571]
 [0.680263]]
2025-04-24 17:14:20,116 | INFO     |: Update Step
2025-04-24 17:14:20,123 | INFO     |:      | Update Time: 0.005 secs
2025-04-24 17:14:20,126 | INFO     |: Bel index     : (np.int64(2), np.int64(2), np.int64(9)) with prob = 1.0
2025-04-24 17:14:20,128 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-24 17:14:20,130 | INFO     |: Belief        : (-0.914, -0.610, 10.000)


In [13]:
ble.disconnect()

2025-04-24 17:14:23,414 | INFO     |: Disconnected from C0:C3:E4:25:23:51


In [ ]:
waypoints = [[-4, -3], [-2, -1], [1, -1], [2, -3], [5, -3], [5, -2], [5, 3], [0, 3], [0, 0]]

In [14]:
point00 = [[0.000, 0.000, -10.000], [0.000, 0.000, -10.000], [0.000, 0.000, -10.000]]

In [23]:
point03 = [[0.000, 0.914, -10.000], [0.000, 0.914, -10.000], [0.000, 0.914, -10.000]]

In [16]:
point_3_2 = [[-0.610, -0.610, -10.000], [-0.914, -0.610, 10.000], [-0.914, -0.610, 10.000]]

In [7]:
point53 = [[1.524, 0.9140, -30.000], [1.524, 0.610, -30.000], [1.524, 0.914, -30.000]]

In [31]:
point5_3 = [[1.524, -0.914, -10.000], [0.914, -1.219, -50.000], [1.524, -0.914, -10.000]]

In [8]:
loc.plot_pose(point53[0], point53[1], point53[2], [1.53, 0.92])